In [17]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold, RepeatedKFold
from scipy import sparse
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
from datetime import datetime

In [18]:
#导入数据
train_abbr=pd.read_csv("datalab/231702/happiness_train_abbr.csv",encoding='ISO-8859-1')
train=pd.read_csv("datalab/231702/happiness_train_complete.csv",encoding='ISO-8859-1')
test_abbr=pd.read_csv("datalab/231702/happiness_test_abbr.csv",encoding='ISO-8859-1')
test=pd.read_csv("datalab/231702/happiness_test_complete.csv",encoding='ISO-8859-1')
test_sub=pd.read_csv("datalab/231702/happiness_submit.csv",encoding='ISO-8859-1')

In [19]:
#观察数据大小
test.shape

(2968, 139)

In [20]:
test_sub.shape

(2968, 2)

In [21]:
train.shape

(8000, 140)

In [22]:
#简单查看数据
train.head()

,id,happiness,survey_type,province,city,county,survey_time,gender,birth,nationality,religion,religion_freq,edu,edu_other,edu_status,edu_yr,income,political,join_party,floor_area,property_0,property_1,property_2,property_3,property_4,property_5,property_6,property_7,property_8,property_other,height_cm,weight_jin,health,health_problem,depression,hukou,hukou_loc,media_1,media_2,media_3,media_4,media_5,media_6,leisure_1,leisure_2,leisure_3,leisure_4,leisure_5,leisure_6,leisure_7,leisure_8,leisure_9,leisure_10,leisure_11,leisure_12,socialize,relax,learn,social_neighbor,social_friend,socia_outing,equity,class,class_10_before,class_10_after,class_14,work_exper,work_status,work_yr,work_type,work_manage,insur_1,insur_2,insur_3,insur_4,family_income,family_m,family_status,house,car,invest_0,invest_1,invest_2,invest_3,invest_4,invest_5,invest_6,invest_7,invest_8,invest_other,son,daughter,minor_child,marital,marital_1st,s_birth,marital_now,s_edu,s_political,s_hukou,s_income,s_work_exper,s_work_status,s_work_type,f_birth,f_edu,f_political,f_work_14,m_birth,m_edu,m_political,m_work_14,status_peer,status_3_before,view,inc_ability,inc_exp,trust_1,trust_2,trust_3,trust_4,trust_5,trust_6,trust_7,trust_8,trust_9,trust_10,trust_11,trust_12,trust_13,neighbor_familiarity,public_service_1,public_service_2,public_service_3,public_service_4,public_service_5,public_service_6,public_service_7,public_service_8,public_service_9
0,1,4,1,12,32,59,2015/8/4 14:18,1,1959,1,1,1,11,NaN,4.0,-2.0,20000,1,NaN,45.0,0,1,0,0,0,0,0,0,0,NaN,176,155,3,2,5,5,2.0,4,2,5,5,4,3,1,4,3,1,2,3,4,1,4,5,4,1,2,4,3,3.0,3.0,2,3,3,3,3,1,1,3.0,30.0,1.0,2.0,1,1,1,2,60000.0,2,2,1,2,0,1,0,0,0,0,0,0,0,NaN,1,0,0.0,3,1984.0,1958.0,1984.0,6.0,1.0,5.0,40000.0,5.0,NaN,NaN,-2,4,4,1,-2,4,1,1,3,2,4,3,50000.0,4,2,-8,-8,5,3,2,3,4,3,-8,4,1,4,50,60,50,50,30.0,30,50,50,50
1,2,4,2,18,52,85,2015/7/21 15:04,1,1992,1,1,1,12,NaN,4.0,2013.0,20000,1,NaN,110.0,0,0,0,0,1,0,0,0,0,NaN,170,110,5,4,3,1,1.0,2,2,1,3,5,1,2,3,4,3,5,4,3,2,3,4,5,1,2,4,3,6.0,2.0,1,3,6,4,8,5,1,3.0,2.0,1.0,3.0,1,1,1,1,40000.0,3,4,1,2,0,1,0,0,0,0,0,0,0,NaN,0,0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1972,3,1,2,1973,3,1,2,1,1,4,2,50000.0,5,4,4,3,5,3,3,3,2,3,3,3,2,3,90,70,70,80,85.0,70,90,60,60
2,3,4,2,29,83,126,2015/7/21 13:24,2,1967,1,0,3,4,NaN,4.0,-2.0,2000,1,NaN,120.0,0,1,1,0,0,0,0,0,0,NaN,160,122,4,4,5,1,1.0,2,2,2,5,1,3,1,4,4,3,5,4,4,2,3,5,5,5,3,4,2,2.0,5.0,2,4,5,4,6,3,2,NaN,NaN,NaN,NaN,1,1,2,2,8000.0,3,3,1,2,0,1,0,0,0,0,0,0,0,NaN,0,2,1.0,3,1990.0,1968.0,1990.0,3.0,1.0,1.0,6000.0,3.0,NaN,NaN,-2,1,1,2,-2,1,1,2,2,1,4,2,80000.0,3,3,3,3,4,3,3,3,3,3,-8,3,1,4,90,80,75,79,80.0,90,90,90,75
3,4,5,2,10,28,51,2015/7/25 17:33,2,1943,1,1,1,3,NaN,4.0,1959.0,6420,1,NaN,78.0,0,0,0,1,0,0,0,0,0,NaN,163,170,4,4,4,1,2.0,2,1,1,5,1,1,1,5,2,4,5,4,5,1,1,5,5,5,2,4,4,1.0,6.0,1,4,5,5,7,2,4,NaN,NaN,NaN,NaN,2,2,2,2,12000.0,3,3,1,1,0,1,0,0,0,0,0,0,0,NaN,1,4,0.0,7,1960.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2,14,1,2,-2,1,1,2,2,1,3,2,10000.0,3,3,4,3,5,3,3,5,4,3,3,3,2,3,100,90,70,80,80.0,90,90,80,80
4,5,4,1,7,18,36,2015/8/10 9:50,2,1994,1,1,1,12,NaN,1.0,2014.0,-1,2,NaN,70.0,0,0,0,0,1,0,0,0,0,NaN,165,110,5,5,3,2,3.0,1,3,4,2,5,5,3,3,3,2,4,4,3,5,2,5,5,1,4,3,4,7.0,5.0,3,2,1,1,1,4,6,NaN,NaN,NaN,NaN,1,2,2,2,-2.0,4,3,1,1,0,1,0,0,0,0,0,0,0,NaN,0,0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970,6,1,10,1972,4,1,15,3,2,3,-8,200000.0,4,3,3,3,5,5,3,4,3,3,3,3,2,2,50,50,50,50,50.0,50,50,50,50


In [23]:
#查看数据是否缺失
train.info(verbose=True,null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 140 columns):
id                      8000 non-null int64
happiness               8000 non-null int64
survey_type             8000 non-null int64
province                8000 non-null int64
city                    8000 non-null int64
county                  8000 non-null int64
survey_time             8000 non-null object
gender                  8000 non-null int64
birth                   8000 non-null int64
nationality             8000 non-null int64
religion                8000 non-null int64
religion_freq           8000 non-null int64
edu                     8000 non-null int64
edu_other               3 non-null object
edu_status              6880 non-null float64
edu_yr                  6028 non-null float64
income                  8000 non-null int64
political               8000 non-null int64
join_party              824 non-null float64
floor_area              8000 non-null float64
proper

In [24]:
#查看label分布
y_train_=train["happiness"]
y_train_.value_counts()

 4    4818
 5    1410
 3    1159
 2     497
 1     104
-8      12
Name: happiness, dtype: int64

In [25]:
#将-8换成3
y_train_=y_train_.map(lambda x:3 if x==-8 else x)


In [26]:
#让label从0开始
y_train_=y_train_.map(lambda x:x-1)

In [27]:
#train和test连在一起
data = pd.concat([train,test],axis=0,ignore_index=True)

In [28]:
#全部数据大小
data.shape

(10968, 140)

In [29]:
#处理时间特征
data['survey_time'] = pd.to_datetime(data['survey_time'],format='%Y-%m-%d %H:%M:%S')
data["weekday"]=data["survey_time"].dt.weekday
data["year"]=data["survey_time"].dt.year
data["quarter"]=data["survey_time"].dt.quarter
data["hour"]=data["survey_time"].dt.hour
data["month"]=data["survey_time"].dt.month

In [30]:
#把一天的时间分段
def hour_cut(x):
    if 0<=x<6:
        return 0
    elif  6<=x<8:
        return 1
    elif  8<=x<12:
        return 2
    elif  12<=x<14:
        return 3
    elif  14<=x<18:
        return 4
    elif  18<=x<21:
        return 5
    elif  21<=x<24:
        return 6

    
data["hour_cut"]=data["hour"].map(hour_cut)

In [31]:
#做问卷时候的年龄
data["survey_age"]=data["year"]-data["birth"]

In [32]:
#让label从0开始
data["happiness"]=data["happiness"].map(lambda x:x-1)

In [33]:
#去掉三个缺失值很多的
data=data.drop(["edu_other"], axis=1)
data=data.drop(["happiness"], axis=1)
data=data.drop(["survey_time"], axis=1)

In [34]:
#是否入党
data["join_party"]=data["join_party"].map(lambda x:0 if pd.isnull(x)  else 1)

In [35]:
#出生的年代
def birth_split(x):
    if 1920<=x<=1930:
        return 0
    elif  1930<x<=1940:
        return 1
    elif  1940<x<=1950:
        return 2
    elif  1950<x<=1960:
        return 3
    elif  1960<x<=1970:
        return 4
    elif  1970<x<=1980:
        return 5
    elif  1980<x<=1990:
        return 6
    elif  1990<x<=2000:
        return 7
    
data["birth_s"]=data["birth"].map(birth_split)
   

In [36]:
#收入分组
def income_cut(x):
    if x<0:
        return 0
    elif  0<=x<1200:
        return 1
    elif  1200<x<=10000:
        return 2
    elif  10000<x<24000:
        return 3
    elif  24000<x<40000:
        return 4
    elif  40000<=x:
        return 5
 

    
data["income_cut"]=data["income"].map(income_cut)

In [37]:
#填充数据
data["edu_status"]=data["edu_status"].fillna(5)
data["edu_yr"]=data["edu_yr"].fillna(-2)
data["property_other"]=data["property_other"].map(lambda x:0 if pd.isnull(x)  else 1)
data["hukou_loc"]=data["hukou_loc"].fillna(1)
data["social_neighbor"]=data["social_neighbor"].fillna(8)
data["social_friend"]=data["social_friend"].fillna(8)
data["work_status"]=data["work_status"].fillna(0)
data["work_yr"]=data["work_yr"].fillna(0)
data["work_type"]=data["work_type"].fillna(0)
data["work_manage"]=data["work_manage"].fillna(0)
data["family_income"]=data["family_income"].fillna(-2)
data["invest_other"]=data["invest_other"].map(lambda x:0 if pd.isnull(x)  else 1)

In [38]:
#填充数据
data["minor_child"]=data["minor_child"].fillna(0)
data["marital_1st"]=data["marital_1st"].fillna(0)
data["s_birth"]=data["s_birth"].fillna(0)
data["marital_now"]=data["marital_now"].fillna(0)
data["s_edu"]=data["s_edu"].fillna(0)
data["s_political"]=data["s_political"].fillna(0)
data["s_hukou"]=data["s_hukou"].fillna(0)
data["s_income"]=data["s_income"].fillna(0)
data["s_work_exper"]=data["s_work_exper"].fillna(0)
data["s_work_status"]=data["s_work_status"].fillna(0)
data["s_work_type"]=data["s_work_type"].fillna(0)

In [39]:
 data=data.drop(["id"], axis=1)

In [40]:
X_train_ = data[:train.shape[0]]
X_test_  = data[train.shape[0]:]

In [41]:
target_column = 'happiness'
feature_columns=list(X_test_.columns) 
feature_columns

['birth',
 'car',
 'city',
 'class',
 'class_10_after',
 'class_10_before',
 'class_14',
 'county',
 'daughter',
 'depression',
 'edu',
 'edu_status',
 'edu_yr',
 'equity',
 'f_birth',
 'f_edu',
 'f_political',
 'f_work_14',
 'family_income',
 'family_m',
 'family_status',
 'floor_area',
 'gender',
 'health',
 'health_problem',
 'height_cm',
 'house',
 'hukou',
 'hukou_loc',
 'inc_ability',
 'inc_exp',
 'income',
 'insur_1',
 'insur_2',
 'insur_3',
 'insur_4',
 'invest_0',
 'invest_1',
 'invest_2',
 'invest_3',
 'invest_4',
 'invest_5',
 'invest_6',
 'invest_7',
 'invest_8',
 'invest_other',
 'join_party',
 'learn',
 'leisure_1',
 'leisure_10',
 'leisure_11',
 'leisure_12',
 'leisure_2',
 'leisure_3',
 'leisure_4',
 'leisure_5',
 'leisure_6',
 'leisure_7',
 'leisure_8',
 'leisure_9',
 'm_birth',
 'm_edu',
 'm_political',
 'm_work_14',
 'marital',
 'marital_1st',
 'marital_now',
 'media_1',
 'media_2',
 'media_3',
 'media_4',
 'media_5',
 'media_6',
 'minor_child',
 'nationality',
 'nei

In [42]:
X_train = np.array(X_train_)
y_train = np.array(y_train_)
X_test  = np.array(X_test_)

In [43]:
X_train.shape

(8000, 145)

In [44]:
y_train.shape

(8000,)

In [45]:
X_test.shape

(2968, 145)

In [46]:
#自定义评价函数
def myFeval(preds, xgbtrain):
    label = xgbtrain.get_label()
    score = mean_squared_error(label,preds)
    return 'myFeval',score

In [47]:
##### xgb

xgb_params = {"booster":'gbtree','eta': 0.005, 'max_depth': 5, 'subsample': 0.7, 
              'colsample_bytree': 0.8, 'objective': 'reg:linear', 'eval_metric': 'rmse', 'silent': True, 'nthread': 8}
folds = KFold(n_splits=5, shuffle=True, random_state=2018)
oof_xgb = np.zeros(len(train))
predictions_xgb = np.zeros(len(test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_+1))
    trn_data = xgb.DMatrix(X_train[trn_idx], y_train[trn_idx])
    val_data = xgb.DMatrix(X_train[val_idx], y_train[val_idx])
    
    watchlist = [(trn_data, 'train'), (val_data, 'valid_data')]
    clf = xgb.train(dtrain=trn_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200, verbose_eval=100, params=xgb_params,feval = myFeval)
    oof_xgb[val_idx] = clf.predict(xgb.DMatrix(X_train[val_idx]), ntree_limit=clf.best_ntree_limit)
    predictions_xgb += clf.predict(xgb.DMatrix(X_test), ntree_limit=clf.best_ntree_limit) / folds.n_splits
    
print("CV score: {:<8.8f}".format(mean_squared_error(oof_xgb, y_train_)))

fold n°1
[0]	train-rmse:2.49558	valid_data-rmse:2.48122	train-myFeval:6.22793	valid_data-myFeval:6.15644
Multiple eval metrics have been passed: 'valid_data-myFeval' will be used for early stopping.

Will train until valid_data-myFeval hasn't improved in 200 rounds.
[100]	train-rmse:1.61256	valid_data-rmse:1.60231	train-myFeval:2.60035	valid_data-myFeval:2.56741
[200]	train-rmse:1.1147	valid_data-rmse:1.11398	train-myFeval:1.24256	valid_data-myFeval:1.24096
[300]	train-rmse:0.851318	valid_data-rmse:0.865434	train-myFeval:0.724743	valid_data-myFeval:0.748976
[400]	train-rmse:0.720895	valid_data-rmse:0.751765	train-myFeval:0.519689	valid_data-myFeval:0.565151
[500]	train-rmse:0.657091	valid_data-rmse:0.704047	train-myFeval:0.431769	valid_data-myFeval:0.495682
[600]	train-rmse:0.623614	valid_data-rmse:0.68494	train-myFeval:0.388894	valid_data-myFeval:0.469143
[700]	train-rmse:0.603522	valid_data-rmse:0.67688	train-myFeval:0.364239	valid_data-myFeval:0.458167
[800]	train-rmse:0.589378	vali

In [48]:
##### lgb

param = {'boosting_type': 'gbdt',
         'num_leaves': 20,
         'min_data_in_leaf': 20, 
         'objective':'regression',
         'max_depth':6,
         'learning_rate': 0.01,
         "min_child_samples": 30,
         
         "feature_fraction": 0.8,
         "bagging_freq": 1,
         "bagging_fraction": 0.8 ,
         "bagging_seed": 11,
         "metric": 'mse',
         "lambda_l1": 0.1,
         "verbosity": -1}
folds = KFold(n_splits=5, shuffle=True, random_state=2018)
oof_lgb = np.zeros(len(X_train_))
predictions_lgb = np.zeros(len(X_test_))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_+1))
   # print(trn_idx)
   # print(".............x_train.........")
   # print(X_train[trn_idx])
  #  print(".............y_train.........")
  #  print(y_train[trn_idx])
    trn_data = lgb.Dataset(X_train[trn_idx], y_train[trn_idx])
    
    val_data = lgb.Dataset(X_train[val_idx], y_train[val_idx])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=200, early_stopping_rounds = 100)
    oof_lgb[val_idx] = clf.predict(X_train[val_idx], num_iteration=clf.best_iteration)
    
    predictions_lgb += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.8f}".format(mean_squared_error(oof_lgb, y_train_)))

fold n°1
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 0.437687	valid_1's l2: 0.470205
[400]	training's l2: 0.372293	valid_1's l2: 0.450994
[600]	training's l2: 0.332259	valid_1's l2: 0.445522
[800]	training's l2: 0.300817	valid_1's l2: 0.442831
[1000]	training's l2: 0.27579	valid_1's l2: 0.441431
Early stopping, best iteration is:
[998]	training's l2: 0.276023	valid_1's l2: 0.441317
fold n°2
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 0.43259	valid_1's l2: 0.495096
[400]	training's l2: 0.367749	valid_1's l2: 0.46997
[600]	training's l2: 0.328506	valid_1's l2: 0.463062
[800]	training's l2: 0.298321	valid_1's l2: 0.460339
[1000]	training's l2: 0.273404	valid_1's l2: 0.458299
Early stopping, best iteration is:
[1032]	training's l2: 0.269784	valid_1's l2: 0.45801
fold n°3
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 0.42951	valid_1's l2: 0.499433
[400]	training's l2: 0.3652

In [51]:
#安装catboost的包
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple catboost

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
    100% |████████████████████████████████| 60.6MB 632kB/s 
jupyter-kernel-gateway 2.3.0 has requirement notebook<6.0,>=5.7.6, but you'll have notebook 5.4.1 which is incompatible.
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [52]:
from catboost import Pool, CatBoostRegressor
# cat_features=[0,2,3,10,11,13,15,16,17,18,19]
from sklearn.model_selection import train_test_split


#X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_train_, y_train_, test_size=0.3, random_state=2019)
# train_pool = Pool(X_train_s, y_train_s,cat_features=[0,2,3,10,11,13,15,16,17,18,19])
# val_pool = Pool(X_test_s, y_test_s,cat_features=[0,2,3,10,11,13,15,16,17,18,19])
# test_pool = Pool(X_test_ ,cat_features=[0,2,3,10,11,13,15,16,17,18,19]) 


kfolder = KFold(n_splits=5, shuffle=True, random_state=2019)
oof_cb = np.zeros(len(X_train_))
predictions_cb = np.zeros(len(X_test_))
kfold = kfolder.split(X_train_, y_train_)
fold_=0
#X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_train, y_train, test_size=0.3, random_state=2019)
for train_index, vali_index in kfold:
    print("fold n°{}".format(fold_))
    fold_=fold_+1
    k_x_train = X_train[train_index]
    k_y_train = y_train[train_index]
    k_x_vali = X_train[vali_index]
    k_y_vali = y_train[vali_index]
    cb_params = {
         'n_estimators': 100000,
         'loss_function': 'RMSE',
         'eval_metric':'RMSE',
         'learning_rate': 0.05,
         'depth': 5,
         'use_best_model': True,
         'subsample': 0.6,
         'bootstrap_type': 'Bernoulli',
         'reg_lambda': 3
    }
    model_cb = CatBoostRegressor(**cb_params)
    #train the model
    model_cb.fit(k_x_train, k_y_train,eval_set=[(k_x_vali, k_y_vali)],verbose=100,early_stopping_rounds=50)
    oof_cb[vali_index] = model_cb.predict(k_x_vali, ntree_end=model_cb.best_iteration_)
    predictions_cb += model_cb.predict(X_test_, ntree_end=model_cb.best_iteration_) / kfolder.n_splits



print("CV score: {:<8.8f}".format(mean_squared_error(oof_cb, y_train_)))


fold n°0
0:	learn: 2.8478482	test: 2.8272440	best: 2.8272440 (0)	total: 58.8ms	remaining: 1h 37m 58s
100:	learn: 0.6914511	test: 0.6811736	best: 0.6811736 (100)	total: 978ms	remaining: 16m 7s
200:	learn: 0.6574334	test: 0.6690809	best: 0.6690809 (200)	total: 1.84s	remaining: 15m 14s
300:	learn: 0.6308319	test: 0.6659466	best: 0.6659466 (300)	total: 2.72s	remaining: 15m 1s
400:	learn: 0.6098269	test: 0.6652447	best: 0.6652182 (399)	total: 3.6s	remaining: 14m 54s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.6652078252
bestIteration = 423

Shrink model to first 424 iterations.
fold n°1
0:	learn: 2.8403088	test: 2.8594981	best: 2.8594981 (0)	total: 12.8ms	remaining: 21m 20s
100:	learn: 0.6913238	test: 0.6960271	best: 0.6960271 (100)	total: 1.06s	remaining: 17m 27s
200:	learn: 0.6597004	test: 0.6750782	best: 0.6750782 (200)	total: 1.96s	remaining: 16m 10s
300:	learn: 0.6342951	test: 0.6674972	best: 0.6674972 (300)	total: 2.83s	remaining: 15m 38s
400:	learn: 0.6134630	

In [53]:
from sklearn import linear_model
# 将lgb和xgb和ctb的结果进行stacking
train_stack = np.vstack([oof_lgb,oof_xgb,oof_cb]).transpose()
test_stack = np.vstack([predictions_lgb, predictions_xgb,predictions_cb]).transpose()


folds_stack = RepeatedKFold(n_splits=5, n_repeats=2, random_state=2018)
oof_stack = np.zeros(train_stack.shape[0])
predictions = np.zeros(test_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds_stack.split(train_stack,y_train)):
    print("fold {}".format(fold_))
    trn_data, trn_y = train_stack[trn_idx], y_train[trn_idx]
    val_data, val_y = train_stack[val_idx], y_train[val_idx]
    
    clf_3 = linear_model.BayesianRidge()
    #clf_3 =linear_model.Ridge()
    clf_3.fit(trn_data, trn_y)
    
    oof_stack[val_idx] = clf_3.predict(val_data)
    predictions += clf_3.predict(test_stack) / 10
    
print("CV score: {:<8.8f}".format(mean_squared_error(oof_stack, y_train_)))


fold 0
fold 1
fold 2
fold 3
fold 4
fold 5
fold 6
fold 7
fold 8
fold 9
CV score: 0.45341306


In [54]:
result=list(predictions)
result=list(map(lambda x: x + 1, result))
test_sub["happiness"]=result
test_sub.to_csv("submit_20190515.csv", index=False)

In [4]:
# # 查看数据文件目录  list datalab files
# !ls datalab/

231702


In [2]:
# # 查看个人永久空间文件  list files in your permanent storage
# !ls /home/tianchi/myspace/
# test_sub.shape

In [ ]:
# # 查看当前kernel下已安装的包  list packages
# !pip list --format=columns

In [ ]:
# # 绘图案例 an example of matplotlib
# %matplotlib inline
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.special import jn
# from IPython.display import display, clear_output
# import time
# x = np.linspace(0,5)
# f, ax = plt.subplots()
# ax.set_title("Bessel functions")

# for n in range(1,10):
#     time.sleep(1)
#     ax.plot(x, jn(x,n))
#     clear_output(wait=True)
#     display(f)

# # close the figure at the end, so we don't get a duplicate
# # of the last plot
# plt.close()